# Sensor Data - Log Data Processing

## Load data and browse data

#### Load data 

In [50]:
import pandas as pd
import os
import json
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [91]:
# Read each line of data as JSON file 
data = []
with open("../data/sensorswww_data.txt", "r") as f:
    for line in f:
        data.append(json.loads(line))

In [92]:
# check dimension of data
len(data)

75092

#### browse global keys

In [93]:
global_keys = list()

In [94]:
for i in range(len(data)):
    for key in data[i].keys():
        if key in global_keys:
            continue
        global_keys.append(key)

In [95]:
global_keys

['distinct_id',
 'lib',
 'properties',
 'type',
 'event',
 '_nocache',
 'time',
 'jssdk_error']

##### global_keys

1. distinct_id: unique identifier   
for registered user:user account;   
for unregistered: CookieID, deviceID, etc.  


2. lib: JavaScript SDK related feature  


3. properties: contains event specific variables in a dictionary format  


4. type:  
profile_set_once: Used for first time activate to set the user's first activation time and first registration time, ignored when profile already exist.   
track: Record an event  


5. event: `$pageview`, `btnClick`, `index_leave`, `demo_leave`, `courses_leave`, `courses_play_leave`, `formSubmit`,  `click_send_cellphone`, `verify_cellphone_code`, `clickSubmit`, `page_close`  

  funnel analysis: to identify the bottleneck of conversion  
  page view -> btnClick -> click_send_cellphone -> verify_cellphone_code -> clickSubmit -> formSubmit 
  
  
6. nocache: A simplied identifier for internal use  


7. time: Exact time occuring of event, measured in miliseconds  


8. day: convert time to days using timestamp  



  



#### check ‘type’ 

In [96]:
# check type
tp = []
for i in range(len(data)):
    tp.append(data[i].get("type",None))
tp = pd.DataFrame({"type":tp})
print(tp["type"].value_counts())

track               65673
profile_set_once     9419
Name: type, dtype: int64


There are 9,419 'profile_set_once' and 65,673 'track'. We are only interested in the 'track' record.

#### check 'event' for 'track' type records 

In [97]:
# check the 'track' reords attribute 'event'
event = []
for i in range(len(data)):
    if data[i]['type'] == 'track':
        event.append(data[i].get("event",None))
event = pd.DataFrame({"event":event})
event_count = event["event"].value_counts()
print(event_count)
print("records with missing values", event.isnull().sum()) 

$pageview                32620
btnClick                 13866
index_leave              10394
demo_leave                3411
about_leave               1032
courses_leave              906
formSubmit                 791
courses_play_leave         747
click_send_cellphone       600
verify_cellphone_code      563
clickSubmit                513
page_close                 230
Name: event, dtype: int64
records with missing values event    0
dtype: int64


Funnel Analysis:   
page view(32620) -> btnClick(13866) -> click_send_cellphone(600) -> verify_cellphone_code(563) -> clickSubmit(513) -> formSubmit(791)   
It seems that most users did not attempt to submit a form

#### check ‘properties' for 'track' type records

In [99]:
# check the 'track' reords attribute 'properties'
event_properties = []
for i in range(len(data)):
    if data[i]['type'] == 'track':
        for property in data[i]['properties'].keys():
            if property in event_properties:
                continue
            event_properties.append(property)
event_properties

['$os',
 '$model',
 '$os_version',
 '$screen_height',
 '$screen_width',
 '$lib',
 '$lib_version',
 '$browser',
 '$browser_version',
 '$latest_referrer',
 '$latest_referrer_host',
 '$latest_utm_source',
 '$latest_utm_medium',
 '$latest_utm_campaign',
 '$latest_utm_content',
 '$latest_utm_term',
 '_latest_ch',
 '_session_referrer',
 '_session_referrer_host',
 'session_page_url',
 'pageUrl',
 'pageStayTime',
 'pagePosition',
 '$is_first_day',
 '$is_first_time',
 '$ip',
 '$referrer',
 '$referrer_host',
 '$url',
 '$url_path',
 '$title',
 'ch',
 'page',
 'name',
 'requestBtn',
 '$utm_source',
 '$utm_medium',
 '$utm_campaign',
 '$utm_content',
 '$utm_term',
 'info',
 'result',
 'contact',
 'verification_code',
 'company',
 'email',
 'site_url',
 'from_url',
 'project_name',
 'isSuccess',
 'isMsg',
 'referrerUrl',
 'referrHostUrl',
 'siteUrl',
 'url_path',
 '_session_referrer_domain',
 '_session_from_url']

#### Select columns we want to get from the text and create pandas Dataframe

In [100]:
# define function to transform milliseconds 'time' into time stamp 'date'
def to_date(epoch):
    return datetime.datetime.fromtimestamp(epoch/1000).strftime("%Y-%m-%d")
# first convert from milliseconds to seconds, and then convert format from time stamp

In [101]:
# define pandas Dataframe with the columns we want to get from the text
tp = []
nocache = []
dist_id = []
event = []
day = []
page_stayTime = []
title = []
latest_referrer = []
latest_referrer_host = []
latest_utm_campaign = []
latest_utm_content = []
latest_utm_medium = []
latest_utm_source = []
latest_utm_term = []
browser = []
browser_version = []
ip = []
model = []
os = []
os_version = []
is_first_day = []
is_first_time = []
name = []
isSuccess = []
isMsg = []
result = []

In [102]:
# Assign NA when the key value not found 
na = None

for i in range(len(data)):
    tp.append(data[i].get('type',na))
    nocache.append(data[i].get('_nocache',na)) 
    dist_id.append(data[i]['distinct_id'])
    event.append(data[i].get('event',na))
    day.append(to_date(data[i]['time']))
    # convert from milliseconds to seconds, and then convert format from time stamp
    page_stayTime.append(data[i]['properties'].get('pageStayTime',na))
    # convert from milliseconds to seconds
    title.append(data[i]['properties'].get('$title',na))
    # page name users view
    latest_referrer.append(data[i]['properties'].get('$latest_referrer',na))    
    # latest refereral website
    latest_referrer_host.append(data[i]['properties'].get('$latest_referrer_host',na))    
    # host website for latest referrel
    latest_utm_campaign.append(data[i]['properties'].get('$latest_utm_campaign',na))    
    # Campaign name
    latest_utm_content.append(data[i]['properties'].get('$latest_utm_content',na))
    # Campaign Content
    latest_utm_medium.append(data[i]['properties'].get('$latest_utm_medium',na))
    # Campaign medium
    latest_utm_source.append(data[i]['properties'].get('$latest_utm_source',na))
    # Campaign source
    latest_utm_term.append(data[i]['properties'].get('$latest_utm_term',na))
    # Campaign term
    browser.append(data[i]['properties'].get('$browser',na)) 
    browser_version.append(data[i]['properties'].get('$browser_version',na)) 
    ip.append(data[i]['properties'].get('$ip',na))    
    model.append(data[i]['properties'].get('$model',na)) # type of device(pc, iphone5, etc...)  
    os.append(data[i]['properties'].get('$os',na))
    os_version.append(data[i]['properties'].get('$os_version',na))
    is_first_day.append(data[i]['properties'].get('$is_first_day',na)) 
    is_first_time.append(data[i]['properties'].get('$is_first_time',na)) 
    name.append(data[i]['properties'].get('name',na))
    # name of buttons
    isSuccess.append(data[i]['properties'].get('isSuccess',na))
    # is the request button successful?
    isMsg.append(data[i]['properties'].get('isMsg',na))
    # is message been sent?
    result.append(data[i]['properties'].get('result',na))
    # status of click_send_cellphone and verify_cellphone_code     

In [103]:
# generate pandas dataframe
df = pd.DataFrame({
'tp': tp,
'nocache': nocache,
'dist_id': dist_id,
'event': event,
'day': day,
'page_stayTime': page_stayTime,
'title': title,
'latest_referrer': latest_referrer,
'latest_referrer_host': latest_referrer_host,
'latest_utm_campaign': latest_utm_campaign,
'latest_utm_content': latest_utm_content,
'latest_utm_medium': latest_utm_medium,
'latest_utm_source': latest_utm_source,
'latest_utm_term': latest_utm_term,
'browser': browser,
'browser_version': browser_version,
'ip': ip,
'model': model,
'os': os,
'os_version': os_version,
'is_first_day': is_first_day,
'is_first_time': is_first_time,
'name': name,    
'isSuccess': isSuccess,
'isMsg': isMsg,
'result': result
})

#### Filter 'track' record.

In [104]:
df['tp'].value_counts()

track               65673
profile_set_once     9419
Name: tp, dtype: int64

profile_set_once: Used for first time activate to set the user's first activation time and first registration time, ignored when profile already exist.   
track: Record an event.  
We will focus only on the 65,673 'track' record.

In [105]:
# filter 'track' record
df = df[df['tp'] == 'track']

#### Save to csv file

In [106]:
data_all = '../data/data_all.csv'
df.to_csv(data_all, index=False)